<h1 style="text-align: center; vertical-align: middle;">Numerical Methods in Accelerator Physics</h1>
<h2 style="text-align: center; vertical-align: middle;">Python examples <span style="color:darkred"> and tasks</span> -- Week 5</h2>

<h2>Run this first!</h2>

Imports and modules:

In [ ]:
from config5 import np, plt, plot_rfwave, tqdm, trange
from scipy.constants import m_p, e, c
import PyNAFF
%matplotlib inline

If the progress bar by `tqdm` (`trange`) later in this document does not work, run this:

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

<h2 style="color:darkred">Tasks: Particle tracking in the CERN Proton Synchrotron</h2>

<h3>Machine parameters for this exercise</h3>

The CERN Proton Synchrotron (PS):
- has a circumference of 2π·100m
- takes protons from the PS Booster at a kinetic energy of 2GeV corresponding to a $\gamma=3.13$
- injects with 50kV of rf voltage, up to 200kV for ramp
- runs at harmonic $h=7$
- has a momentum compaction factor of $\alpha_c=0.027$
- typical acceleration rate of (up to) $\dot{B}=2$ T/s, the bending radius is $\rho=70.08$ m

<h3 style="color: darkred">Tasks based on topics of week 4</h3>

<div style="color:darkred; margin-top: 2em;">Based on the CERN Proton Synchrotron (PS) machine parameters (see above): <br />

1. Track particles given a stationary synchronous particle ($\varphi_s=0$).
2. Determine the frequency of the synchrotron motion (via NAFF).
3. Track particles given an accelerating synchronous particle ($\varphi_s>0$).
4. Compute the transition energy $\gamma_{\text{t}}$ for CERN PS.
4. Set machine energy ($\gamma$) above transition / relativistic regime, track like (4) again.

$\implies$ take note of the observed trajectories in phase space (for the following)!
</div>
<!-- 
3. Derivation of Hamiltonian $\mathcal{H}(z,\delta)=T(\delta) + V(z)$ for stationary synchronous particle ($\varphi_s=0$) -->

<h3>Helper functions and classes</h3>

Some convenience functions to compute the speed β and the relativistic Lorentz factor γ:

In [ ]:
def beta(gamma: float) -> float:
    '''Speed β in units of c from relativistic Lorentz factor γ.'''
    return np.sqrt(1 - gamma**-2)

def gamma(p: float) -> float:
    '''Relativistic Lorentz factor γ from total momentum p.'''
    return np.sqrt(1 + (p / (mass * c))**2)

def p_from_gamma(gamma: float) -> float:
    '''Total momentum p from relativistic Lorentz factor γ.'''
    return mass * c * np.sqrt(gamma**2 - 1)

We gather all the machine parameters in a class named `Machine`.

A `Machine` instance knows
- at which energy the synchronous particle (reference γ, `gamma_ref`, or alternatively the momentum `p0()`) currently runs,
- what the acceleration rate is in terms of the synchronous phase φ_s, `phi_s`
- how to compute the phase-slip factor $\eta$, `eta`, for a particle at a certain momentum p0 + Δp
- how to update the energy of the synchronous particle via `update_gamma_ref()` when a turn has passed

In [ ]:
charge = e
mass = m_p

class Machine(object):
    gamma_ref = 3.13
    circumference = 2*np.pi*100
    voltage = 50e3
    harmonic = 7
    alpha_c = 0.027
    phi_s = 0
    
    def __init__(self, gamma_ref: float = gamma_ref, circumference: float = circumference,
                 voltage: float = voltage, harmonic: int = harmonic, 
                 alpha_c: float = alpha_c, phi_s: float = phi_s) -> None:
        '''Override default settings by giving explicit arguments.'''
        self.gamma_ref = gamma_ref
        self.circumference = circumference
        self.voltage = voltage
        self.harmonic = harmonic
        self.alpha_c = alpha_c
        self.phi_s = phi_s
    
    def eta(self, deltap: float) -> float:
        '''Phase-slip factor for a particle.'''
        p = self.p0() + deltap
        return self.alpha_c - gamma(p)**-2

    def p0(self) -> float:
        '''Momentum of synchronous particle.'''
        return self.gamma_ref * beta(self.gamma_ref) * mass * c

    def update_gamma_ref(self) -> None:
        '''Advance the energy of the synchronous particle
        according to the synchronous phase by one turn.
        '''
        deltap_per_turn = charge * self.voltage / (
            beta(self.gamma_ref) * c) * np.sin(self.phi_s)
        new_p0 = self.p0() + deltap_per_turn
        self.gamma_ref = gamma(new_p0)

To compute a synchronous phase, you may use these convenience functions to compute the $\arcsin$ on the interval of [-π/2,π/2] .

Remember you may likely want to find a synchronous phase on $\varphi_s\in[0,π]$ !

In [ ]:
def deltap_per_turn(Bdot: float, rho: float, circumference: float, p0: float) -> float:
    Trev = circumference / (beta(gamma(p0)) * c)
    return Bdot * rho * charge * Trev

In [ ]:
def compute_phi_s(deltap_per_turn: float, p0: float, voltage: float) -> float:
    '''Return *first* positive phase which matches the
    given Δp/turn on the interval [0, π/2].
    Do check whether you need to use π-φ_s for stability!
    '''
    return np.arcsin(
        deltap_per_turn * beta(gamma(p0)) * c / (charge * voltage)
    )

<h3>Tracking</h3>

The tracking equations for the longitudinal plane read:

$$\begin{cases}\,
    z_{n+1} &= z_n - \eta C \left(\cfrac{\Delta p}{p_0}\right)_n \\
    (\Delta p)_{n+1} &= (\Delta p)_n + \cfrac{q V}{(\beta c)_n}\cdot\left(\sin\left(\varphi_s - \cfrac{2\pi}{C}\cdot hz_{n+1}\right) - \sin(\varphi_s)\right)
\end{cases}$$

<p style="color:darkred">Please implement them with a leapfrog (half-drift + kick + half-drift) scheme.</p>

In [ ]:
def track_one_turn(z_n, deltap_n, machine):
    m = machine # all machine parameters are provided by m.circumference etc
    
    # half drift
    z_nhalf = ... ## to be implemented, you can use m.eta(deltap_n) and m.p0()
    
    # rf kick
    deltap_n1 = ... ## to be implemented, you can use the helper functions beta(gamma) and gamma(m.p0))

    # update reference energy
    m.update_gamma_ref()
    
    # half drift
    z_n1 = ... ## to be implemented
    return z_n1, deltap_n1

<h4>1. Track particles given a stationary synchronous particle</h4>
The `Machine` instance will keep track of the reference energy during the tracking by calling `update_gamma_ref()` once per turn:

In [ ]:
m = Machine()

Particles are tracked by their two longitudinal coordinates $(z, \Delta p)$. The initial values are stored in `z_ini` and `deltap_ini` as `numpy.array`s. These should have `N` entries for $N$ particles.

(You may use numpy helper functions such as `np.linspace` or `np.arange` for convenient initialisation!)

<p style="color:darkred">Please initialise a particle distribution with 40 particles at $z=0$ and evenly spaced $\Delta p$ over the intervall $\left[0,0.01\cdot\textrm{m.p0()}\right]$.</p>

In [ ]:
n_turns = 3000
deltap_ini = ... ## to be implemented
z_ini = np.zeros_like(deltap_ini)

In [ ]:
N = len(z_ini)
assert (N == len(deltap_ini))

To store the coordinate values during tracking, prepare some `n_turns` long 2D arrays with `N` entries per turn:

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

We would also like to store the reference gamma for each turn:

In [ ]:
gammas = np.zeros(n_turns, dtype=np.float64)
gammas[0] = m.gamma_ref

<p style="color:darkred">Let's go, here you can add the tracking loop over the number of turns `n_turns`!</p>

In [ ]:
for i_turn in range(1, n_turns):
    z[i_turn], deltap[i_turn] = ... ## to be implemented using the function track_one_turn(z_n, deltap_n, machine)
    gammas[i_turn] = m.gamma_ref

As a first check, we plot the reference energy over all turns (should be constant for a stationary synchronous particle).

In [ ]:
plt.plot(gammas)
plt.xlabel('Turns')
plt.ylabel('$\gamma_{ref}$')

Let's have a look on the phase space evolution:

In [ ]:
plt.scatter(z, deltap / m.p0(), marker='.', s=0.5)
plt.xlim(-50,100)
plt.xlabel('$z$ [m]')
plt.ylabel('$\Delta p/p_0$')

<h4>2. Determine the frequency of the synchrotron motion (via NAFF).</h4>

In order to visualize the oscillations, we plot the particle positions over turns:

In [ ]:
plt.plot(((z.T)[:19]).T)
plt.xlabel('turns')
plt.ylabel('$z$ [m]')

<p style="color:darkred">Please implement how to determine the spectrum using NAFF.</p>

In [ ]:
freqs_naff = []

... ## to be implemented (compare nmap3.ipynb)

freqs_naff = np.array(freqs_naff)

The results are plotted:

In [ ]:
plt.plot(np.max(((z.T)[:19]).T,axis=0), freqs_naff, c='r', marker='.')

#plt.xticks([-np.pi/2, 0, np.pi/2, np.pi], [r"$-\pi/2$", "0", r"$\pi/2$", r"$\pi$"])
plt.xlabel(r'Maximum amplitude z [m]')
plt.ylabel('Phase advance / \n' + r'integration step $\Delta t$ [$2\pi$]');

<h4>3. Track particles given an accelerating synchronous particle ($\gamma=3.13$, typical acceleration rate of $\dot{B}=2$ T/s, bending radius $\rho=70.08$ m, rf voltage 200kV)</h4>

<p style="color:darkred">For the machine initialisation, we need to know the synchronous phase for the given acceleration ramp. Please calculate this phase using the function `compute_phi_s()` above.</p>

In [ ]:
phi_s=compute_phi_s( ... ) ## arguments to be implemented
phi_s

Now we can initialize the machine and set up the initial phase space:

In [ ]:
m = Machine(phi_s=phi_s)

In [ ]:
n_turns = 1000
deltap_ini = np.linspace(0, 0.01 * m.p0(), 40)#np.array([0.]) #np.linspace(start, end, 20)
z_ini = np.zeros_like(deltap_ini)

In [ ]:
N = len(z_ini)
assert (N == len(deltap_ini))

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
gammas = np.zeros(n_turns, dtype=np.float64)
gammas[0] = m.gamma_ref

Tracking like before:

In [ ]:
for i_turn in range(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)
    gammas[i_turn] = m.gamma_ref

The reference energy should now increase according to the acceleration ramp:

In [ ]:
plt.plot(gammas)
plt.xlabel('Turns')
plt.ylabel('$\gamma_{ref}$')

And the resulting phase space looks like this:

In [ ]:
plt.scatter(z, deltap / m.p0(), marker='.', s=0.5)
plt.xlabel('$z$ [m]')
plt.ylabel('$\Delta p/p_0$')

<h4 style="color:darkred">4. Compute the transition energy for CERN PS</h4>

In [ ]:
gamma_t = ... ## to be implemented
gamma_t

<h4>5. Set machine energy above transition / relativistic regime, track again.</h4>

<b>For the same synchronous phase we expect longitudinal defocusing of the particle distribution. Let's show this in the tracking.</b>

Same steps as before: machine and phase space initialization, tracking and plotting:

In [ ]:
m = Machine(gamma_ref=7,phi_s=phi_s)

In [ ]:
n_turns = 5000
deltap_ini = np.linspace(0, 0.01 * m.p0(), 40)#np.array([0.]) #np.linspace(start, end, 20)
z_ini = np.zeros_like(deltap_ini)

In [ ]:
N = len(z_ini)
assert (N == len(deltap_ini))

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
gammas = np.zeros(n_turns, dtype=np.float64)
gammas[0] = m.gamma_ref

In [ ]:
for i_turn in range(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)
    gammas[i_turn] = m.gamma_ref

In [ ]:
plt.plot(gammas)
plt.xlabel('Turns')
plt.ylabel('$\gamma_{ref}$')

In [ ]:
plt.scatter(z, deltap / m.p0(), marker='.', s=0.5)
plt.xlabel('$z$ [m]')
plt.ylabel('$\Delta p/p_0$')

As expected, all particles are on unstable trajectories.<br /><br />
<b style="color:darkred">Let's change the synchronous phase to longitudinal focusing!</b><br />
<span style="color:darkred">What synchronous phase is required for this?</span>

In [ ]:
m = Machine(gamma_ref=7,phi_s= ... )

In [ ]:
n_turns = 10000
deltap_ini = np.linspace(0, 0.01 * m.p0(), 40)#np.array([0.]) #np.linspace(start, end, 20)
z_ini = np.zeros_like(deltap_ini)

In [ ]:
N = len(z_ini)
assert (N == len(deltap_ini))

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
gammas = np.zeros(n_turns, dtype=np.float64)
gammas[0] = m.gamma_ref

In [ ]:
for i_turn in range(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)
    gammas[i_turn] = m.gamma_ref

In [ ]:
plt.plot(gammas)
plt.xlabel('Turns')
plt.ylabel('$\gamma_{ref}$')

In [ ]:
plt.scatter(z, deltap / m.p0(), marker='.', s=0.5)
plt.xlim(right=100)
plt.ylim(bottom=-0.02)
plt.xlabel('$z$ [m]')
plt.ylabel('$\Delta p/p_0$')

<h2>Visualization of synchrotron tune by particle tracking (slide 17)</h2>

You can override the default PS parameters by supplying arguments to the `Machine(...)` instantiation, e.g. change the `gamma_ref` by `m = Machine(gamma_ref=20)` or the `phi_s` etc.

In [ ]:
m = Machine(voltage=200e3,phi_s=0.456)

Now we define the kinetic and potential energy terms and the Hamiltonian function(al) itself:

In [ ]:
def T(deltap, machine):
    '''Kinetic energy term in Hamiltonian.'''
    return -0.5 * machine.eta(deltap) / machine.p0() * deltap**2

def U(z, machine, beta_=None):
    '''Potential energy term in Hamiltonian.
    If beta is not given, compute it from synchronous particle.
    '''
    m = machine
    if beta_ is None:
        beta_ = beta(gamma(m.p0()))
    ampl = charge * m.voltage / (beta_ * c * 2 * np.pi * m.harmonic)
    phi = m.phi_s - 2 * np.pi * m.harmonic / m.circumference * z
    # convenience: define z at unstable fixed point
    z_ufp = -m.circumference * (np.pi - 2 * m.phi_s) / (2 * np.pi * m.harmonic)
    # convenience: offset by potential value at unstable fixed point
    # such that unstable fixed point (and separatrix) have 0 potential energy
    return ampl * (-np.cos(phi) + 
                   2 * np.pi * m.harmonic / m.circumference * (z - z_ufp) * np.sin(m.phi_s) +
                   -np.cos(m.phi_s))

In [ ]:
def hamiltonian(z, deltap, machine):
    return T(deltap, machine) + U(z, machine, beta_=beta(gamma(machine.p0() + deltap)))

In [ ]:
def plot_hamiltonian(machine, zleft=-50, zright=50, dpmax=0.01, cbar=True):
    '''Plot Hamiltonian contours across (zleft, zright) and (-dpmax, dpmax).'''
    Z, DP = np.meshgrid(np.linspace(zleft, zright, num=1000), 
                        np.linspace(-dpmax, dpmax, num=1000))
    H = hamiltonian(Z, DP * machine.p0(), machine) / machine.p0()
    
    plt.contourf(Z, DP, H, cmap=plt.get_cmap('hot_r'), levels=12,
                 zorder=0, alpha=0.5)
    plt.xlabel('$z$ [m]')
    plt.ylabel(r'$\delta$')
    if cbar:
        colorbar = plt.colorbar(label=r'$\mathcal{H}(z,\Delta p)\,/\,p_0$')
        colorbar.ax.axhline(0, lw=2, c='b')
    plt.contour(Z, DP, H, colors='b', linewidths=2, levels=[0])

Let's plot the Hamiltonian landscape on phase space:

In [ ]:
plot_hamiltonian(m)

Tracking just like last lecture and the exercise above:

In [ ]:
n_turns = 1000
deltap_ini = np.linspace(0, 0.01 * m.p0(), 100)
z_ini = np.zeros_like(deltap_ini)

N = len(z_ini)
assert (N == len(deltap_ini))

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

Record the evolution of energy $\gamma_\mathrm{ref}$ and also Hamiltonian values of particles during tracking:

In [ ]:
gammas = np.zeros(n_turns, dtype=np.float64)
gammas[0] = m.gamma_ref

H_values = np.zeros_like(z)
H_values[0] = hamiltonian(z_ini, deltap_ini, m) / m.p0()

The tracking loop:

In [ ]:
for i_turn in range(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)
    gammas[i_turn] = m.gamma_ref
    H_values[i_turn] = hamiltonian(z[i_turn], deltap[i_turn], m) / m.p0()

In [ ]:
plt.plot(gammas)
plt.xlabel('Turns')
plt.ylabel('$\gamma_{ref}$');

In [ ]:
plot_hamiltonian(m, zleft=-150, zright=50, dpmax=0.017)
plt.scatter(z[:500], deltap[:500] / m.p0(), marker='.', s=0.5)
plt.xlabel('$z$ [m]')
plt.ylabel('$\Delta p/p_0$')

In [ ]:
plt.plot(H_values, c='C0');
plt.xlabel('Turns')
plt.ylabel(r'$\mathcal{H}(z,\Delta p)\,/\,p_0$');

Helper function to plot rf wave, potential and longitudinal phase space in one figure:

In [ ]:
def plot_rf_overview():
    z_range = np.linspace(-150, 40, num=1000)
    # z location of unstable fixed point:
    z_ufp = -m.circumference * (np.pi - 2 * m.phi_s) / (2 * np.pi * m.harmonic)

    fig, ax = plt.subplots(3, 1, figsize=(6, 10), sharex=True)

    plt.sca(ax[0])
    plt.plot(z_range, 1e-3 * m.voltage * np.sin(m.phi_s - 2 * np.pi * m.harmonic / m.circumference * z_range))
    plt.axhline(0, c='gray', lw=2)
    plt.axhline(1e-3 * m.voltage * np.sin(m.phi_s), c='purple', lw=2, ls='--')
    plt.axvline(0, c='purple', lw=2)
    plt.axvline(z_ufp, c='red', lw=2)
    plt.ylabel('rf wave $V(z)$ [kV]')

    plt.sca(ax[1])
    plt.plot(z_range, 1e6 * U(z_range, m) / m.p0())
    plt.axhline(0, c='gray', lw=2)
    plt.ylabel(r'$U(z)\,/\,p_0\cdot 10^6$')

    plt.scatter([z_ufp], [0], marker='*', c='white', edgecolor='red', zorder=10)
    plt.scatter([0], [U(0, m) / m.p0()], marker='d', c='white', edgecolor='purple', zorder=10)

    plt.sca(ax[2])
    plot_hamiltonian(m, zleft=z_range[0], zright=z_range[-1], cbar=False)
    plt.scatter([z_ufp], [0], marker='*', c='white', edgecolor='red', zorder=10)
    plt.scatter([0], [0], marker='d', c='white', edgecolor='purple')
    plt.xlabel('$z$ [m]')
    plt.ylabel('$\delta$')
    plt.subplots_adjust(hspace=0)
    
    return fig, ax

In [ ]:
plot_rf_overview();

In the plot, stable and unstable fix points are marked in purple and red, respectively.



<h3 style="color: darkred;">Task: Determine the synchrotron tune using NAFF and compare to the result of the Paper&Pen exercise</h3>

In [ ]:
... ## plot oscillations for comparison

In [ ]:
... ## implement frequency calculation by NAFF

In [ ]:
... ## plot spectrum and compare to Paper&Pen result

<h2>Phase Space Initialisation</h2>

<h3>Linear Congruential Generator (slide 20)</h3>

In [ ]:
class RandomNumberGenerator(object):
    def __init__(self, M, a, c, seed):
        self.M = M
        self.a = a
        self.c = c
        self.xk = seed

    def generate(self):
        xk1 = (self.a * self.xk + self.c) % self.M
        self.xk = xk1
        return xk1 / self.M

Instantiate the linear congruential generator by Lewis et al with a certain `seed`:

In [ ]:
prng_standard = RandomNumberGenerator(
    M=2**31 - 1, 
    a=7**5, 
    c=0, 
    seed=12345)

Generate a set of numbers from the sequence and analyse:

In [ ]:
results = [prng_standard.generate() for i in range(10000)]

In [ ]:
plt.hist(results)
plt.xlabel('$x$')
plt.ylabel('#draws');

In [ ]:
plt.scatter(results[:-1], results[1:], s=1, marker='.')
plt.xlabel('$x_k$')
plt.ylabel('$x_{k+1}$');

<p style="color: darkred;">$\implies$ What happens when you change the parameters $M,a,c$?<br /><br />
Try e.g. $a=5$ or $M=2^{31}-2$...</p>

In [ ]:
prng_standard2 = RandomNumberGenerator(
    M=2**31 - 2, 
    a=7**5, 
    c=0, 
    seed=12345)

In [ ]:
results2 = [prng_standard2.generate() for i in range(10000)]

In [ ]:
plt.hist(results2)
plt.xlabel('$x$')
plt.ylabel('#draws');

In [ ]:
plt.scatter(results2[:-1], results2[1:], s=1, marker='.')
plt.xlabel('$x_k$')
plt.ylabel('$x_{k+1}$');

In [ ]:
prng_standard3 = RandomNumberGenerator(
    M=2**31 - 1, 
    a=5, 
    c=0, 
    seed=12345)

In [ ]:
results3 = [prng_standard3.generate() for i in range(10000)]

In [ ]:
plt.hist(results3)
plt.xlabel('$x$')
plt.ylabel('#draws');

In [ ]:
plt.scatter(results3[:-1], results3[1:], s=1, marker='.')
plt.xlabel('$x_k$')
plt.ylabel('$x_{k+1}$');

<h3>Box-Muller method (slide 21)</h3>

In [ ]:
prng_1 = RandomNumberGenerator(
    M=2**31 - 1, 
    a=7**5, 
    c=0, 
    seed=12345)

prng_2 = RandomNumberGenerator(
    M=2**31 - 1, 
    a=7**5, 
    c=0, 
    seed=42)

In [ ]:
def generate_normal():
    xi1 = prng_1.generate()
    xi2 = prng_1.generate()
    r = np.sqrt(-2 * np.log(xi2))
    x = r * np.cos(2 * np.pi * xi1)
    y = r * np.sin(2 * np.pi * xi1)
    return x, y

In [ ]:
results = np.array(
    [generate_normal() for i in range(10000)]
).flatten()

In [ ]:
plt.hist(results, bins=20);

<h3>NumPy has it all...</h3>

The `numpy` library implements all of these (based on a better behaved variant of the linear congruential generator):

In [ ]:
plt.hist(np.random.random(size=10000));
plt.xlabel('$x$')
plt.ylabel('#draws');

In [ ]:
plt.hist(np.random.normal(size=10000), bins=20);
plt.xlabel('$x$')
plt.ylabel('#draws');

<h3>Initalization of longitudinal particle distribution: Interactive Tracking (slide 25)</h3>

Initialise a bi-Gaussian distribution in the longitudinal phase-space plane for tracking!

Refer once more to the CERN PS scenario, below transition and in a stationary rf bucket:

In [ ]:
m = Machine(gamma_ref=3.13, phi_s=0)

The length of the rf bucket corresponds to $C/h$:

In [ ]:
m.circumference / m.harmonic

We choose an rms bunch length of $\sigma_z=10\,$m:

In [ ]:
sigma_z = 10

The corresponding rms momentum difference $\sigma_{\Delta p}$ is given through the equilibrium condition (equal Hamiltonian values):

In [ ]:
sigma_deltap = np.sqrt(
    2 * m.p0() / -m.eta(0) * 
    charge * m.voltage * np.pi * m.harmonic / (beta(gamma(m.p0())) * c * m.circumference**2)
) * sigma_z

In [ ]:
sigma_dp = sigma_deltap / m.p0()
sigma_dp

In [ ]:
plot_hamiltonian(m)
plt.scatter([sigma_z, 0], [0, sigma_dp], marker='*', c='k');

Back to tracking, out of the box:

In [ ]:
N = 1000
n_turns = 5000

In [ ]:
np.random.seed(12345)

z_ini = np.random.normal(loc=0, scale=sigma_z, size=N)
deltap_ini = np.random.normal(loc=0, scale=sigma_deltap, size=N)

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

Now let's return to the outlined Monte-Carlo approach, analysing the results in terms of statistical moments.

First the centroid:

In [ ]:
plt.plot(np.mean(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\langle z \rangle$');

Then the rms beam size (bunch length):

In [ ]:
plt.plot(np.std(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\sigma_z$');

<h2>What happened?</h2>

Let's look at the generated initial distribution of macro-particles:

In [ ]:
plot_hamiltonian(m);
plt.scatter(z[0], deltap[0] / m.p0(), marker='.', s=1);

$\implies$ particles have been generated outside the rf bucket! 

<h2>Rejection Sampling Method: Cut particles outside of the separatrix</h2>

A solution to the problem of generating particles outside the separatrix: <i>reject</i> them at generation!

$\leadsto$ a word of caution: this approach modifies the effective distribution function (and the correspondingly generated effective rms values $\sigma_z, \sigma_\delta$ become smaller)! 

In [ ]:
np.random.seed(12345)

z_ini = np.random.normal(loc=0, scale=sigma_z, size=N)
deltap_ini = np.random.normal(loc=0, scale=sigma_deltap, size=N)

<h2>Rejection Sampling Method: Resample these particles</h2>

Hamiltonian values of particles outside separatrix are positive (below transition), $\mathcal{H}>0$ (using the full nonlinear Hamiltonian)!

NB: Due to the discrete kicks in the finite difference maps, the Hamiltonian is only an approximation for the separatrix: better remain at a few percent distance inside of it!

In [ ]:
H_safetymargin = 0.05 * hamiltonian(0, 0, m)

H_values = hamiltonian(z_ini, deltap_ini, m) - H_safetymargin

while any(H_values >= 0):
    mask_bad = H_values >= 0
    N_bad = np.sum(mask_bad)
    print (N_bad)
    # re-initialise bad particles:
    z_ini[mask_bad] = np.random.normal(loc=0, scale=sigma_z, size=N_bad)
    deltap_ini[mask_bad] = np.random.normal(loc=0, scale=sigma_deltap, size=N_bad)
    # re-evaluate rejection condition
    H_values = hamiltonian(z_ini, deltap_ini, m) - H_safetymargin

Now we should be good to go!

In [ ]:
plot_hamiltonian(m);
plt.scatter(z_ini, deltap_ini / m.p0(), marker='.', s=1);

Tracking again...

In [ ]:
z = np.zeros((n_turns, N), dtype=np.float64)
deltap = np.zeros_like(z)

z[0] = z_ini
deltap[0] = deltap_ini

In [ ]:
for i_turn in trange(1, n_turns):
    z[i_turn], deltap[i_turn] = track_one_turn(z[i_turn - 1], deltap[i_turn - 1], m)

Looking again at the centroid:

In [ ]:
plt.plot(np.mean(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\langle z \rangle$');

Then the rms bunch length:

In [ ]:
plt.plot(np.std(z, axis=1))

plt.xlabel('Turns')
plt.ylabel(r'$\sigma_z$');

<p style="color: darkred;">$\implies$ try larger choices of initial $\sigma_z$ and see where the bunch length evolution saturates.</p>

$\leadsto$ another problem with initialising Gaussian distributions: at larger amplitudes $z$, the small-amplitude approximation with $\mathcal{H}_\mathrm{stat,small}$ necessarily breaks down! A Gaussian particle distribution is <b>not</b> in equilibrium for sufficiently large rms values in a nonlinear potential, the particles will <b>filament</b>! (...and the rms emittance will grow, as one can observe in the final equilibrium rms bunch length which is larger than the initial $\sigma_z$!)

$\implies$ generally require full nonlinear Hamiltonian $\mathcal{H}$ to construct PDF

$$\psi(\mathcal{H})\propto\exp\left(\cfrac{\mathcal{H}}{\mathcal{H}_0}\right)$$

<h2>RMS Emittance</h2>

Define statistical <b>rms emittance</b> as in Lecture 2:

In [ ]:
def emittance(z, deltap):
    N = len(z)
    
    # subtract centroids
    z = z - 1/N * np.sum(z)
    deltap = deltap - 1/N * np.sum(deltap)
    
    # compute Σ matrix entries
    z_sq = 1/N * np.sum(z * z)
    deltap_sq = 1/N * np.sum(deltap * deltap)
    crossterm = 1/N * np.sum(z * deltap)
    
    # determinant of Σ matrix
    epsilon = np.sqrt(z_sq * deltap_sq - crossterm * crossterm)
    return epsilon

In [ ]:
epsn_z = np.array([emittance(z_i, deltap_i) for z_i, deltap_i in zip(z, deltap)])

In [ ]:
plt.plot(epsn_z / e)

plt.xlabel('Turns')
plt.ylabel('$\epsilon_z$ [eV.s]');

<p style="color: darkred;">$\implies$ try larger choices of initial $\sigma_z$ and observe how the emittance evolves.</p>